In [ ]:
import sqlite3

from IPython.core.magic import register_cell_magic

from tabulate import tabulate
import sqlparse

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


def run_and_print(sql):
    try:
        cursor.execute(sql)
    except sqlite3.OperationalError as e:
        print("Error:\n", sql)
        raise e

    conn.commit()

    rows = cursor.fetchall()
    if cursor.description is None:
        return

    print(tabulate(rows, headers=[d[0] for d in cursor.description]), end='\n\n')


@register_cell_magic
def sql(_, cell):
    run_and_print("DROP TABLE IF EXISTS trace")
    run_and_print("CREATE TABLE trace(trace TEXT)")
    for sql in sqlparse.split(cell):
        run_and_print(sql)
    run_and_print("SELECT * FROM trace")

In [ ]:
%%sql

INSERT INTO trace VALUES ('Reticulating splines...');

CREATE TABLE signal(
    clk BOOLEAN,
    instr INT
    );
INSERT INTO signal VALUES (
    FALSE,
    0
    );

CREATE TABLE register(
    R0 INT,
    R1 INT,
    R2 INT,
    R3 INT,
    R4 INT,
    R5 INT,
    R6 INT,
    R7 INT,
    PC INT,
    COND INT
    );
INSERT INTO register VALUES (
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    );

CREATE TABLE memory(
    address INT,
    value INT
    );

pragma recursive_triggers = 1;


In [ ]:
%%sql

DROP TRIGGER IF EXISTS clk_trigger_rising;
CREATE TRIGGER clk_trigger_rising
AFTER UPDATE OF clk ON signal
    WHEN OLD.clk = 0 AND NEW.clk = 1
BEGIN
    -- Insert into trace table based on clk edge detection
    INSERT INTO trace VALUES ('clk rising edge');
END;

DROP TRIGGER IF EXISTS clk_trigger_falling;
CREATE TRIGGER clk_trigger_falling
AFTER UPDATE OF clk ON signal
    WHEN OLD.clk = 1 AND NEW.clk = 0
BEGIN
    -- Insert into trace table based on clk edge detection
    INSERT INTO trace VALUES ('clk falling edge');

    -- Read instruction from memory based on PC
    -- There are two bytes the High and Low. We are Big Endian.
    INSERT INTO trace VALUES ('reading instruction from memory...');
    UPDATE signal
        SET instr = (
            (SELECT value FROM memory WHERE address = (SELECT PC FROM register)) << 8 |
            (SELECT value FROM memory WHERE address = (SELECT PC FROM register) + 1)
        );

    -- Update PC in register table based on falling edge detection
    INSERT INTO trace VALUES ('incrementing PC...');
    UPDATE register
        SET PC = PC + 2;
END;

DROP TRIGGER IF EXISTS instr_trigger;
CREATE TRIGGER instr_trigger
AFTER UPDATE OF instr ON signal
BEGIN
    -- Insert into trace table based on instruction update
    INSERT INTO trace VALUES ('instruction updated');
END;

DROP TRIGGER IF EXISTS instr_hlt_trigger;
CREATE TRIGGER instr_hlt_trigger
AFTER UPDATE OF instr ON signal
    WHEN NEW.instr = 0xF025
BEGIN
    -- Insert into trace table based on instruction update
    INSERT INTO trace VALUES ('HLT instruction detected');
END;

In [ ]:
%%sql

insert into memory values (0x3200, 0xF0);
insert into memory values (0x3201, 0x25);
update register set PC = 0x31FA;

In [ ]:
%%sql
-- drive the clock signal
UPDATE signal SET clk = 1;
UPDATE signal SET clk = 0;

select * from register;
select * from signal;